In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from math import sin, cos, sqrt, atan2, radians
import re
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [2]:
calgary_df = pd.read_csv('data/calgary_listings.csv')
calgary_df.head()

,price,postal_code,address,mls,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
0,219900,T2Y 3G2,", 22 Shawbrooke Court SW",A1045249,2.0,2.0,1.0,1109.0,townhouse,NaN,NaN,NaN,NaN,NaN,950.87,na,1.0,0.0
1,524000,T2T 2A9,1 - 1540 34 AVE SW,A1045048,2.0,2.0,1.0,1346.0,townhouse,NaN,NaN,NaN,NaN,NaN,2265.84,na,0.0,1.0
2,419000,T3E 1Y6,1 - 1911 25A ST SW,A1018871,2.0,1.0,1.0,1128.0,townhouse,NaN,NaN,NaN,NaN,NaN,1811.81,na,0.0,1.0
3,285000,T2S 0L1,1 - 210 25 AVE SW,A1017109,2.0,1.0,NaN,849.0,condo,NaN,NaN,1998.0,22.0,NaN,1232.38,NaN,0.0,1.0
4,374000,T2T 2C5,1 - 2119 34 AVE SW,A1033672,3.0,1.0,1.0,1220.0,townhouse,NaN,NaN,1972.0,48.0,NaN,1617.22,na,0.0,0.0


In [3]:
post_code_list = []

for i in calgary_df["postal_code"]:
    post_code_list.append(i)

In [4]:
scores_walk = []
scores_bike = []
scores_transit = []

for i in post_code_list:

    time.sleep(5)
    
    try:
        postal_code = i.replace(" ", "%20")
        url_score = "https://www.walkscore.com/score/" + str(postal_code)
        time.sleep(5)

        # Parse HTML with Beautiful Soup
        response = requests.get(url_score)
        code_soup = BeautifulSoup(response.text, 'html.parser')

        if 'pp.walk.sc/badge/walk/score' in str(code_soup):
            ws = str(code_soup).split('pp.walk.sc/badge/walk/score/')[1][:2].replace('.','')
            scores_walk.append(ws)
        else:
            ws = 0
            scores_walk.append(ws)
        if 'pp.walk.sc/badge/bike/score' in str(code_soup):
            bs = str(code_soup).split('pp.walk.sc/badge/bike/score/')[1][:2].replace('.','')
            scores_bike.append(bs)
        else:
            bs = 0
            scores_bike.append(bs)
        if 'pp.walk.sc/badge/transit/score' in str(code_soup):
            ts = str(code_soup).split('pp.walk.sc/badge/transit/score/')[1][:2].replace('.','')
            scores_transit.append(ts)
        else:
            ts = 0
            scores_transit.append(ts)
    except:
        ws = 0
        scores_walk.append(ws)
        bs = 0
        scores_bike.append(bs)
        ts = 0
        scores_transit.append(ts)

In [5]:
score_df_trans = {'postal_code':post_code_list, 
                  'walk_score':scores_walk, 
                  'bike_score':scores_bike, 
                  'transit_score':scores_transit}
score_df_dup = pd.DataFrame(score_df_trans)
score_df = score_df_dup.drop_duplicates()
score_df.head()

,postal_code,walk_score,bike_score,transit_score
0,T2Y 3G2,17,62,39
1,T2T 2A9,54,80,42
2,T3E 1Y6,59,93,61
3,T2S 0L1,72,97,58
4,T2T 2C5,75,82,51


In [6]:
score_df.to_csv('data/score_df.csv', index=False)